Implementation of Connect 4 AI with inspirations from the following:
    <p>http://blog.gamesolver.org/solving-connect-four/01-introduction/
    <p>https://github.com/nwestbury/pyConnect4
    <p>https://towardsdatascience.com/creating-the-perfect-connect-four-ai-bot-c165115557b0


In [ ]:
import numpy
from kaggle_environments import make

env = make("connectx", debug=True)
# # None indicates which agent will be manually played.
# env.run(["random", "random"])
# env.render(mode="ipython", width=500, height=450)

In [ ]:
def my_agent3(obs, config):
    import numpy as np
    import random
    import time
    
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)        
#         score = 1e2*num_threes - 1e2*num_threes_opp - 1e9*num_fours_opp + 1e9*num_fours
        score = 10 * num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)        
        score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False
    

    def negamax(node, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
        if depth = 0 or node is a terminal node then
            return color × the heuristic value of node

        childNodes := generateMoves(node)
        childNodes := orderMoves(childNodes)
        value := −∞
        foreach child in childNodes do
            value := max(value, −negamax(child, depth − 1, −β, −α, −color))
            α := max(α, value)
            if α ≥ β then
                break (* cut-off *)
        return alpha
        """
        global pos_count
        pos_count += 1
        is_terminal = is_terminal_node(node, config)
        
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
#         valid_moves = [c for c in range(config.columns) if node[0][c] == 0]  
        valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
#         valid_moves := orderMoves(valid_moves)
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, -negamax(child, depth - 1, mark%2+1, config, -beta, -alpha))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return alpha
    
    
#     def iterative_deepening(grid, obs.mark, config, timeout):
#         best_col, best_score = -1, -np.Inf
#         valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
#         for depth in range(3, 50):
#             for col in valid_moves:
#                 next_grid = drop_piece(grid, col, mark, config)        
#                 score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
#                 if not search_cutoff and score > best_score:
#                     best_col, best_score = col, best_score
#                 else: # Running out of time
#                     break
#         return best_col
        
        
        
        
  
    start = time.time()
    global pos_count
    pos_count = 0
    column_order = []
    for i in range(7):
        column_order.append(7//2 + (1 - 2* (i %2)) * (i+1)//2)
    
#     valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    # Order column from center to side
    valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3
    elif sum(obs.board) == 1 and 3 in valid_moves:
        return 3 # Play middle whether opponent places there or not
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    #Get scores for each branch
    N_STEPS = 4
    
    # Use the heuristic to assign a score to each possible board in the next step
    if len(valid_moves) >= 6:
        N_STEPS=4
    if len(valid_moves) ==5:
        N_STEPS=5
    if len(valid_moves) ==4:
        N_STEPS=6
    if len(valid_moves) <=3:
        N_STEPS=10
        
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))        
    
    #Get the highest score value    
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    col = max_cols[0] # Taking the center columns first
    end = time.time()
#     print("timeout", config.timeout)
    print("my_agent3 excecution time", (end-start), "move", col, "score", scores[col], "at depth", N_STEPS, "pos count", pos_count)
    
    return col

        

In [ ]:
from collections import OrderedDict
import numpy as np
from functools import lru_cache
import pickle
import random
import time
import zlib
import base64 as b64
# from kaggle_environments import make


opening_book = None

class LRUCache:
 
    # initialising capacity
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity
 
    # we return the value of the key
    # that is queried in O(1) and return -1 if we
    # don't find the key in out dict / cache.
    # And also move the key to the end
    # to show that it was recently used.
    def get(self, key: int) -> object:
        if key not in self.cache:
            return None
        else:
            self.cache.move_to_end(key)
            return self.cache[key]
 
    # first, we add / update the key by conventional methods.
    # And also move the key to the end to show that it was recently used.
    # But here we will also check whether the length of our
    # ordered dictionary has exceeded our capacity,
    # If so we remove the first key (least recently used)
    def put(self, key: int, value: object) -> None:
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last = False)

transposition_table = LRUCache(1000000)
# cache.put(1, (-1, 0, 7))
# cache.put(2, (-2, 5, 5))
# print(cache.get(1))

# print(cache.cache)

def serializeAndCompress(value, verbose=True):
    serializedValue = pickle.dumps(value)
    if verbose:
        print('Lenght of serialized object:', len(serializedValue))
    c_data =  zlib.compress(serializedValue, 9)
    if verbose:
        print('Lenght of compressed and serialized object:', len(c_data))
    #   return b64.b64encode(c_data)
    return c_data

def decompressAndDeserialize(compresseData):
    # d_data_byte = b64.b64decode(compresseData)
    # data_byte = zlib.decompress(d_data_byte)
    # d_data_byte = b64.b64decode(compresseData)
    data_byte = zlib.decompress(compresseData)
    value = pickle.loads(data_byte)
    return value

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))

# return a bitmask containg a single 1 corresponding to the top cel of a given column


@lru_cache(maxsize=None)
def top_mask_col(col):
    return (1 << 5) << col * 7

# return a bitmask containg a single 1 corresponding to the bottom cell of a given column


@lru_cache(maxsize=None)
def bottom_mask_col(col):
    return 1 << col * 7

# return a bitmask 1 on all the cells of a given column


@lru_cache(maxsize=None)
def column_mask(col):
    return ((1 << 6) - 1) << col * 7



@lru_cache(maxsize=None)
def bitboardBits(i):
    """"
    Returns the number of bits in a bitboard (7x6).
    Running time: O(1)
    Help from: http://stackoverflow.com/q/9829578/1524592
    """
    i = i & 0xFDFBF7EFDFBF  # magic number to mask to only legal bitboard
    # positions (bits 0-5, 7-12, 14-19, 21-26, 28-33, 35-40, 42-47)
    i = (i & 0x5555555555555555) + ((i & 0xAAAAAAAAAAAAAAAA) >> 1)
    i = (i & 0x3333333333333333) + ((i & 0xCCCCCCCCCCCCCCCC) >> 2)
    i = (i & 0x0F0F0F0F0F0F0F0F) + ((i & 0xF0F0F0F0F0F0F0F0) >> 4)
    i = (i & 0x00FF00FF00FF00FF) + ((i & 0xFF00FF00FF00FF00) >> 8)
    i = (i & 0x0000FFFF0000FFFF) + ((i & 0xFFFF0000FFFF0000) >> 16)
    i = (i & 0x00000000FFFFFFFF) + ((i & 0xFFFFFFFF00000000) >> 32)

    return i


@lru_cache(maxsize=None)
def compute_winning_position(position, mask):
    HEIGHT = 6
    # vertical
    r = (position << 1) & (position << 2) & (position << 3)

    # horizontal
    p = (position << (6+1)) & (position << 2*(6+1))
    r |= p & (position << 3*(6+1))
    r |= p & (position >> (6+1))
    p = (position >> (6+1)) & (position >> 2*(6+1))
    r |= p & (position << (6+1))
    r |= p & (position >> 3*(6+1))

    # diagonal 1
    p = (position << HEIGHT) & (position << 2*HEIGHT)
    r |= p & (position << 3*HEIGHT)
    r |= p & (position >> HEIGHT)
    p = (position >> HEIGHT) & (position >> 2*HEIGHT)
    r |= p & (position << HEIGHT)
    r |= p & (position >> 3*HEIGHT)

    # diagonal 2
    p = (position << (HEIGHT+2)) & (position << 2*(HEIGHT+2))
    r |= p & (position << 3*(HEIGHT+2))
    r |= p & (position >> (HEIGHT+2))
    p = (position >> (HEIGHT+2)) & (position >> 2*(HEIGHT+2))
    r |= p & (position << (HEIGHT+2))
    r |= p & (position >> 3*(HEIGHT+2))

    return r & (279258638311359 ^ mask)

def move_to_col(move):
    """
    Convert move bitmap to column number (zero indexed)
    """
    for i in range(7):
        if move >> 7 * (i+1) == 0:
            return i




class Position(object):
    WIDTH = 7
    HEIGHT = 6
    bottom_mask = 4432676798593
    board_mask = 279258638311359

    def __init__(self, position, mask):
        self.position = position
        self.mask = mask
        # TODO: do this only once, not when creating child positions as we already have moves by then
        self.moves = bitboardBits(self.mask)
        self.best_move = None

    @classmethod
    def get_position_mask_bitmap(self, grid, mark):
        # print("grid", grid)
        # print("mark", mark)
        position, mask = '', ''
        # Start with right-most column
        for j in range(6, -1, -1):
            # Add 0-bits to sentinel
            mask += '0'
            position += '0'
            # Start with bottom row
            for i in range(0, 6):
                mask += ['0', '1'][int(grid[i][j] != 0)]
                position += ['0', '1'][int(grid[i][j] == mark)]
        return int(position, 2), int(mask, 2)

    def key(self):
        return self.position + self.mask

    def partial_key3(self, key, col):
        pos = 1 << (col * 7)
        while pos & self.mask > 0:
            key *= 3
            if pos & self.position > 0:
                key += 1
            else:
                key += 2
            pos <<= 1

        key *= 3

        return key

    def key3(self):
        key_forward = 0
        for i in range(7):
            key_forward = self.partial_key3(key_forward, i)

        key_reverse = 0
        for i in range(6, -1, -1):
            key_reverse = self.partial_key3(key_reverse, i)

        return key_forward // 3 if key_forward < key_reverse else key_reverse // 3

    def can_play(self, col):
        return (self.mask & top_mask_col(col)) == 0

    def play(self, col):
        self.position ^= self.mask
        self.mask |= self.mask + bottom_mask_col(col)
        self.moves += 1

    # Plays a possible move given by its bitmap representation
    # move: a possible move given by its bitmap representation
    # only one bit of the bitmap should be set to 1
    # the move should be a valid possible move for the current player
    def play_move(self, move):
        self.position ^= self.mask
        self.mask |= move
        self.moves += 1

    # Plays a sequence of successive played columns, mainly used to initilize a board.
    def play_seq(self, seq):
        for i in range(len(seq)):
            col = int(seq[i]) - 1
            if col < 0 or col >= 7 or not self.can_play(col) or self.is_winning_move2(col):
                return i
            self.play(col)
        return len(seq)

    def is_winning_move(self, col):
        pos = self.position
        pos |= (self.mask + bottom_mask_col(col)) & column_mask(col)
        return self.has_won(pos)

    def has_drawn(self):
        """
        If the board has all of its valid slots filled, then it is a draw.
        We mask the board to a bitboard with all positions filled
        (0xFDFBF7EFDFBF) and if all the bits are active, it is a draw.
        """
        # return (self.position & 0xFDFBF7EFDFBF) == 0xFDFBF7EFDFBF
        return self.moves == 42

    def pretty_print(self):
        print("position", self.position)
        print("mask", self.mask)
        opp_position = self.position ^ self.mask
        print("board     position  mask      key       bottom")
        print("          0000000   0000000")
        for i in range(5, -1, -1):  # iterate backwards from 5 to 0
            board_row = "".join("x" if (self.position >> i+k*7) & 1 == 1
                                else "o" if (opp_position >> i+k*7) & 1 == 1 else "." for k in range(7))
            pos_row = "".join(str((self.position >> i+j*7) & 1)
                              for j in range(7))
            mask_row = "".join(str((self.mask >> i+j*7) & 1) for j in range(7))
            print(board_row + "   " + pos_row + "   " + mask_row)

    # Bitmap of the next possible valid moves for the current player
    # Including losing moves.

    def possible(self):
        return (self.mask + self.bottom_mask) & self.board_mask

    # counts number of bit set to one in a 64bits integer
    def popcount(self, m):
        c = 0
        while m > 0:
            m &= m - 1
            c += 1
        return c

    # Score a possible move.
    # The score we are using is the number of winning spots
    # the current player has after playing the move.
    def move_score(self, move):
        return self.popcount(compute_winning_position(self.position | move, self.mask))

    # Return a bitmask of the possible winning positions for the opponent
    def opponent_winning_position(self):
        return compute_winning_position(self.position ^ self.mask, self.mask)

    # Return a bitmask of the possible winning positions for the current player
    def winning_position(self):
        return compute_winning_position(self.position, self.mask)

    # Indicates whether the current player wins by playing a given column.
    # This function should never be called on a non-playable column.

    def is_winning_move2(self, col):
        return self.winning_position() & self.possible() & column_mask(col)

    def can_win_next(self):
        return self.winning_position() & self.possible()

    # Return a bitmap of all the possible next moves the do not lose in one turn.
    # A losing move is a move leaving the possibility for the opponent to win directly.
    def possible_non_losing_moves(self):
        possible_mask = self.possible()
        opponent_win = self.opponent_winning_position()
        forced_moves = possible_mask & opponent_win
        if forced_moves:
            if forced_moves & (forced_moves - 1):
                return 0
            else:
                possible_mask = forced_moves
        return possible_mask & ~(opponent_win >> 1)

    # mirrored bitboard
    # 6666666 5555555 4444444 3333333 2222222 1111111 0000000
    # 0000000 0000000 0000000 0000000 0000000 0000000 1111111 column_mask[0]
    # 0000000 0000000 0000000 0000000 0000000 1111111 0000000 column_mask[1]

    @classmethod
    def mirror(self, field):
        mirrored = 0x0
        mirrored |= ((field & column_mask(0)) << 42)
        mirrored |= ((field & column_mask(1)) << 28)
        mirrored |= ((field & column_mask(2)) << 14)
        mirrored |= (field & column_mask(3))
        mirrored |= ((field & column_mask(4)) >> 14)
        mirrored |= ((field & column_mask(5)) >> 28)
        mirrored |= ((field & column_mask(6)) >> 42)
        return mirrored

        # k1 = 0x5555555555555555
        # k2 = 0x3333333333333333
        # k4 = 0x0f0f0f0f0f0f0f0f
        # x = ((x >> 1) & k1) | ((x & k1) << 1)
        # x = ((x >> 2) & k2) | ((x & k2) << 2)
        # x = ((x >> 4) & k4) | ((x & k4) << 4)
        # x = x >> 1
        # return x
        
class Solver(object):

    cache = None

    def __init__(self):
        self.node_count = 0
        self.MIN_SCORE = -18
        self.MAX_SCORE = 18

        self.column_order = []
        for i in range(7):
            self.column_order.append(7 // 2 + (1 - 2 * (i % 2)) * (i + 1) // 2)

        global opening_book
        if opening_book == None:
            print("Loading opening book...")
            opening_book = dict()
            with open("./opening_book.24", 'rb') as f:
                opening_book_data = f.read()
                # opening_book = pickle.load(f)
                opening_book = decompressAndDeserialize(opening_book_data)

    def negamax(self, p: Position, alpha, beta):
        """
        Alpha-beta with Memory to solve a position without depth
        """
        # assert(alpha < beta)
        # assert(not p.can_win_next())
        alpha_orig = alpha
        best_move = p.best_move

        self.node_count += 1    # increment counter of explored nodes

        possible = p.possible_non_losing_moves()
        if possible == 0:   # if no possible non losing moves, opponent wins next move
            return int(-(42 - p.moves) / 2)

        if p.moves >= 40:  # check for draw game
            # print("draw game")
            for i in range(7):
                move = possible & column_mask(self.column_order[i])
                if move != 0:   # don't add none move
                    p.best_move = move
                    break
            return 0

        # lower bound of score as opponent cannot win next move
        min = int(-(40 - p.moves) / 2)
        if alpha < min:
            alpha = min             # there is no need to keep alpha below our min possible score
            # prune the exploration if the [alpha:beta] window is empty
            if alpha >= beta:
                return alpha

        max = (41 - p.moves) // 2
        if beta > max:
            beta = max              # there is no need to keep beta above our max possible score
            # prune the exploration if [alpha:beta] window is empty
            if alpha >= beta:
                return beta

        key = p.key()
        entry = transposition_table.get(key)
        if entry != None:
            val, best_move2 = entry
            if val > self.MAX_SCORE - self.MIN_SCORE + 1:   # we have a lower bound
                min = val + 2 * self.MIN_SCORE - self.MAX_SCORE - 2
                if alpha < min:
                    # here is no need to keep alpha below our min possible score.
                    alpha = min
                    best_move = best_move2    # Setting the current best_move from previous table entry
                    # prune the exploration if the [alpha;beta] window is empty.
                    if alpha >= beta:
                        if best_move != None:
                            p.best_move = best_move # Set best_move before cutting off
                        return alpha

            else:       # we have an upper bound
                max = val + self.MIN_SCORE - 1
                if beta > max:
                    # there is no need to keep beta above our max possible score.
                    beta = max
                    # prune the exploration if the [alpha;beta] window is empty.
                    if alpha >= beta:
                        return beta

        moves = []
        for i in range(7):
            move = possible & column_mask(self.column_order[i])
            if move != 0:   # don't add none move
                moves.append(move)

        moves.sort(reverse=True, key=lambda move: p.move_score(move))
        # This is super important!!! Initializing best_move to the first in moves.
        # It handles the cases where aggressive min/max prunning might have best_move uninitialized otherwise.
        if best_move == None:   
            best_move = moves[0]
        best_score = alpha
        for move in moves:
            p2 = Position(p.position, p.mask)
            # It's opponent turn in P2 position after current player plays x column.
            p2.play_move(move)

            # explore opponent's score within [-beta;-alpha] windows:
            # no need to have good precision for score better than beta (opponent's score worse than -beta)
            # no need to check for score worse than alpha (opponent's score worse better than -alpha)
            score = -self.negamax(p2, -beta, -alpha)
            if score > best_score:
                best_score = score
                best_move = move

            if score >= beta:
                # prune the exploration if we find a possible move better than what we were looking for.
                break

            if score > alpha:
                # reduce the [alpha;beta] window for next exploration, as we only
                # need to search for a position that is better than the best so far.
                alpha = score

        if best_score >= beta:
            # save the lower bound of the position, as well as best_move
            transposition_table.put(
                    key, ((score + self.MAX_SCORE - 2 * self.MIN_SCORE + 2), best_move))
            if best_move != None:
                p.best_move = best_move
            
        elif best_score < alpha_orig:
        # save the upper bound of the position, no need to cache best_move
            transposition_table.put(key, (alpha - self.MIN_SCORE + 1, None))
        if best_move != None:
            p.best_move = best_move
        return best_score

    def solve(self, p: Position, weak=False):

        best_move = None
        # check if win in one move as the Negamax function does not support this case.
        if p.can_win_next():
            for col in range(7):
                if p.is_winning_move2(col) > 0:
                    # print("my_agent_new winning move", col)
                    return int((43 - p.moves + 1) / 2)

        min = int(-(42 - p.moves) / 2)
        max = int((43 - p.moves) / 2)
        if weak:
            min = -1
            max = 1

        while min < max:
            med = min + int((max - min) / 2)
            if med <= 0 and int(min / 2) < med:
                med = int(min / 2)
            elif med >= 0 and int(max / 2) > med:
                med = int(max / 2)
            r= self.negamax(p, med, med + 1)
            
            if r <= med:
                max = r    
            else:
                min = r
                
        return min

    def analyze(self, p: Position, weak=False):
        scores = dict()
        for col in range(7):
            if p.can_play(col):
                # print("Getting score for col", col)
                if p.is_winning_move2(col):
                    scores[col] = (43 - p.moves) // 2
                else:
                    p2 = Position(p.position, p.mask)
                    p2.play(col)
                    scores[col] = -self.solve(p2, weak)
        return scores
                    
def my_agent(obs, config):
    import numpy as np
    import random
    import time
    # import cProfile, pstats, io
    # from pstats import SortKey
    
    
    # with cProfile.Profile() as pr:
    weak = False

    start = time.time()
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    position, mask = Position.get_position_mask_bitmap(grid, obs.mark)
    # print(position, mask)
    p = Position(position, mask)
    solver = Solver()

    
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3

    # If there is a winning move, return it now!
    if p.can_win_next():
        # print("can win next", p.position, p.mask)
        # p.pretty_print()
        for col in range(7):
            if p.is_winning_move2(col) > 0:
                print("my_agent_new winning move", col)
                return col
    
    # If we only store mirrored positions, then if we find something with the same key3
    # We need to check if it's mirrored from the current position. 
    # book storage: key -> move
    # during generation, we use key3 to not store transposed positions
    # at lookup time, if key doesn't exist, mirror the current position and check again if found, return mirrored col
    if p.moves < 27:
        p2 = Position(position, mask)
        key = p2.key()
        if key in opening_book:
            best_move = opening_book[key]
            print("Move", p.moves + 1, "Playing best move from the book", best_move)
            return best_move
        else:
            p3 = Position(Position.mirror(position), Position.mirror(mask))
            key2 = p3.key()
            if key2 in opening_book:
                best_move = opening_book[key2]
                print("Move", p.moves + 1, "Playing mirrored best move from the book", 6 - best_move)
                return 6 - best_move
    
    # scores = solver.analyze(p, weak)
    # max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    # best_move = random.choice(max_cols) # Taking the center columns first
    # if p.moves >= 39:
    #     print("here")
    score = solver.solve(p, weak)
    # print("score", score)
    if p.best_move == None:   # we lost next move, just play any move
        for i in range(7):
            if p.can_play(i):
                best_col = i
                break
    else:
        best_col = move_to_col(p.best_move)

    end = time.time()
    print("my_agent_new move #", p.moves+1, "time", (end-start), "move", best_col, "score", score, "pos count", solver.node_count)
    return best_col

In [ ]:
def test():
    import cProfile, pstats, io
    from pstats import SortKey

    grid = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]]

    position, mask = Position.get_position_mask_bitmap(grid, 1)
    print(position, mask)
    p = Position(position, mask)
    # p.play_seq("4444443")
    p.play_seq("444444326555553233322676235266611177")
    p.pretty_print()
    # position, mask = 35254972727296, 136583988658304
    # p = Position(position, mask)

    # board = [0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0]
    # board = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1]
    # board = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0, 0, 2, 1, 2, 2, 1, 1]
    # board = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1]
    board = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1]
    grid = np.asarray(board).reshape(6, 7)
    print(grid)
    # print(grid)
    position, mask = Position.get_position_mask_bitmap(grid, 1)
    # print(position, mask)
    p = Position(position, mask)
    solver = Solver()


    print("moves made so far", p.moves)
    # count = p.popcount(p.mask)
    # print("count", count)
    # constant = 4
    # p2 = Position(Position.mirror(p.position), Position.mirror(p.mask))
    # p2.pretty_print()
    # 
    # with cProfile.Profile() as pr:
    start = time.time()
    score = solver.solve(p, False)
    print("solved", score, "move", move_to_col(p.best_move), (time.time()-start), "seconds with", solver.node_count, "nodes explored.") #format(move, "0b"))
    
    score = solver.solve(p, False)
    print("solved", score, "move", move_to_col(p.best_move), (time.time()-start), "seconds with", solver.node_count, "nodes explored.") #format(move, "0b"))
    start = time.time()
    scores = solver.analyze(p, False)
    print(scores, time.time()-start, "seconds with", solver.node_count, "nodes explored.")
    # s = io.StringIO()
    # sortby = SortKey.CUMULATIVE  #, SortKey.PCALLS
    # ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    # # ps.dump_stats("./profile.stats")
    # # pr.print_stats()
    # ps.print_stats(50)
    # print(s.getvalue())


In [ ]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# Two random agents play one game round
# env.run([my_agent, "negamax"])
# env.run([my_agent, "random"])
# env.run(["random", my_agent])
# env.run(["negamax", my_agent])
# env.run([my_agent3, my_agent])
# env.run([my_agent3, "negamax"])
env.run([my_agent, my_agent])
# env.play([None, my_agent])

# Show the game
env.render(mode="ipython")

In [ ]:
import random as random
position, mask = 44056576, 132136961

solver = Solver()
p = Position(position, mask)
# print("bit number", bitboardBits(p.mask))
# print("winning pos", p.is_winning_move2(0))
# print("possible non-losing moves", p.possible_non_losing_moves())
p.pretty_print()

# valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
# print(valid_moves)
# scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, 10) for col in valid_moves]))   
# print(scores)
# max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
# col = random.choice(max_cols) # Taking the center columns first
# print("move", col)


In [50]:
def get_win_percentages(agent1, agent2, n_rounds=10):
    from kaggle_environments import make, evaluate
    import numpy as np
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # print("outcomes:", outcomes)
    print("outcomes:", outcomes)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("outcomes:", outcomes)
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=my_agent3, agent2=my_agent)

In [ ]:
import sys
from kaggle_environments import utils, agent

out = sys.stdout
my_agent4 = utils.read_file("./submission_with_dataset.py")
my_agent5 = utils.read_file("./submission_with_dataset_10ply.py")
my_agent6 = utils.read_file("./main_22ply.py")
my_agent7 = utils.read_file("./main_24ply_explore.py")
sys.stdout = out

# env = make("connectx", debug=True)
# env.run([agent, agent])
# env.render(mode="ipython")
# print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed..." + env.state[0].status + env.state[1].status)

In [ ]:
get_win_percentages(agent1=my_agent4, agent2=my_agent)

In [ ]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# Two random agents play one game round
# env.run([my_agent, "negamax"])
# env.run(["negamax", my_agent])
# env.run([my_agent3, my_agent])
# env.run([my_agent3, "negamax"])
# env.run([my_agent, my_agent3])
# env.run([my_agent5, my_agent])
# env.run([my_agent, my_agent5])
# env.run([my_agent6, my_agent])
# env.run([my_agent, my_agent6])
env.run([my_agent7, my_agent])
# env.run([my_agent, my_agent7])

# Show the game
env.render(mode="ipython")

In [ ]:
import time as time
# grid = [
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 2, 0, 0, 0],
#  [0, 0, 0, 1, 0, 0, 0],
#  [0, 0, 0, 0, 2, 1, 1],
#  [0, 0, 2, 1, 1, 2, 1]]

grid = [
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]

position, mask = Position.get_position_mask_bitmap(grid, 1)
print(position, mask)
solver = Solver()
p = Position(position, mask)
# p.play_seq("4444443")
p.play_seq("444444326555553233322676235266611177")
p.pretty_print()
# position, mask = 44056576, 132136961
# p = Position(position, mask)
print("moves made so far", p.moves)
# count = p.popcount(p.mask)
# print("count", count)
# constant = 4
# p2 = Position(Position.mirror(p.position), Position.mirror(p.mask))
# p2.pretty_print()
score, move = solver.solve(p)
print("solved", score, "move", move_to_col(move))
# depth = 10
# print(p.key3())
# print(ternary(p.key3()))
# # print("bit number", bitboardBits(p.mask))
# # print("winning pos", p.is_winning_move2(0))
# # print("possible non-losing moves", p.possible_non_losing_moves())
# start = time.time()
# valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
# print(valid_moves)

# # If there is a winning move, return it now!
# if p.can_win_next():
#     # print("can win next", p.position, p.mask)
#     # p.pretty_print()
#     for col in valid_moves:
#         if p.is_winning_move2(col) > 0:
#             print("my_agent_new winning move", col)
#             # return col

# if p.moves < 12:  # we should have entries in the opening book
#     best_score, best_move = -100, None
#     for col in valid_moves:
#         p2 = Position(p.position, p.mask)
#         p2.play(col)
#         key3 = p2.key3()
#         if key3 in opening_book:
#             # if obs.mark == 2:
#             #     score = -opening_book[key3]
#             # else: 
#             #     score = opening_book[key3]
#             score = -opening_book[key3]
#             # print("found", key3, score, "for", col)
#             if score > best_score:
#                 best_score = score
#                 best_move = col
#         else:
#             print("key", key3, "not found for col", col)
#     if best_move != None:
#         print("Playing best move from the book", best_move)
#         # return best_move
# scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, depth) for col in valid_moves]))   
# print(scores)
# max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())] 
# col = max_cols[0]
# end = time.time()
# print("my_agent_new move #", p.moves+1, "time", (end-start), "move", col, "score", scores[col], "at depth", depth, "pos count", solver.node_count)
 

In [ ]:
# book_raw = "./connect4/12ply.book"
# book_key = "./connect4/12ply_key.book"

# with open(book_raw) as f:
#     with open(book_key, "w") as g:
#         count = 0
#         for line in f:
#             p = Position(0, 0)
#             row = line.split()
#             p.play_seq(row[0])
#             g.write(str(p.key3()) + " " + row[1] + "\n")
#             count += 1
# print(count, "rows written to ", book_key)


24ply book: Positions explored: 3155528 Positions ignore: 2062512
<p>1093016 rows read to dict
<p>Lenght of serialized object: 10763422
<p>Lenght of compressed and serialized object: 3846018

In [ ]:
# import pickle
# import zlib

# book_raw = "./connect4/24ply.book"

# book = dict()
# with open(book_raw) as f:
#     count = 0
#     for line in f:
#         p = Position(0, 0)
#         [moves, val] = line.split()
#         p.play_seq(moves)
#         key = p.key()
#         book[int(key)] = int(val)
#         count += 1
# print(count, "rows read to dict")

# with open('opening_book.24', 'wb') as g:
#     data = serializeAndCompress(book)
#     # pickle.dump(book, g)
#     g.write(data)







In [ ]:
import json
from kaggle_environments.utils import structify

def win_loss_draw(score):
    if score>0: 
        return 'win'
    if score<0: 
        return 'loss'
    return 'draw'

def score(agent, max_lines = 1000):
    #Scores a connect-x agent with the dataset
    print("scoring ",agent)
    count = 0
    good_move_count = 0
    perfect_move_count = 0
    observation = structify({'mark': None, 'board': None})
    with open("./connect4/refmoves1k_kaggle") as f:
        for line in f:
            
            data = json.loads(line)
            moves = data["move score"]
            perfect_score = max(moves)
            
            observation.board = data["board"]
            num_moves = sum(observation.board)
            if abs(perfect_score) <= (43 - num_moves) // 2 - 5:
                continue
            # find out how many moves are played to set the correct mark.
            ply = len([x for x in data["board"] if x>0])
            if ply&1:
                observation.mark = 2
            else:
                observation.mark = 1
            
            #call the agent
            agent_move = agent(observation,env.configuration)
            count += 1
            
            perfect_moves = [ i for i in range(7) if moves[i]==perfect_score]

            if(agent_move in perfect_moves):
                perfect_move_count += 1
                print("Perfect!")
            else:
                print("Not perfect")
                print(observation)

            if win_loss_draw(moves[agent_move]) == win_loss_draw(perfect_score):
                good_move_count += 1
                print("Good!")
            else:
                print("Not Good")

            if count == max_lines:
                break

        print("perfect move percentage: ",perfect_move_count/count)
        print("good moves percentage: ",good_move_count/count)
        print("Total analyzed", count)

In [ ]:
# Score the 2 built in agents
from kaggle_environments import make
env = make("connectx")
# the built in agents are remarkably slow so only evaluating on 100 moves here
# score(env.agents["random"],100)  |
# score(env.agents["negamax"],100)
# score(my_agent, 100)
score(my_agent, 1000)

scoring  <function my_agent at 0x1084c08b0>
<p>my_agent_new move # 11 time 26.101505994796753 move 3 score 11 pos count 2629203
<p>my_agent_new move # 11 time 215.2161102294922 move 1 score 2 pos count 13592695
<p>my_agent_new move # 11 time 88.20222902297974 move 2 score 0 pos count 3826535
<p>my_agent_new move # 11 time 926.2882270812988 move 3 score 1 pos count 24117864
<p>my_agent_new move # 11 time 22.580275058746338 move 5 score -2 pos count 967767
<p>perfect move percentage:  1.0
<p>good moves percentage:  1.0
<p>
<p>After return best_move from negamax directly.
<p>scoring  <function my_agent at 0x10dc81b80>
<p>my_agent_new move # 11 time 0.026532888412475586 move 3 score 11 pos count 1174
<p>my_agent_new move # 11 time 65.1391339302063 move 2 score 2 pos count 4581071
<p>my_agent_new move # 11 time 37.29094409942627 move 2 score 0 pos count 3122981
<p>my_agent_new move # 11 time 168.20349287986755 move 3 score 1 pos count 14370336
<p>my_agent_new move # 11 time 11.442300796508789 move 5 score -2 pos count 963083
<p>perfect move percentage:  1.0
<p>good moves percentage:  1.0

<p>After fixing ways to return best move:
<p>perfect move percentage:  1.0
<p>good moves percentage:  1.0
<p>Total analyzed 358
<p>total time 120.89296579360962 seconds.